# 1. SVM

In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.svm import SVC

# 데이터 로드
data = pd.read_csv("../../../../monunmon.csv") # CSV 파일 경로

# 레이블 이진 분류를 위해 변환 (0~94 -> 1, -1 -> 0)
data['Label'] = data['Label'].apply(lambda x: 1 if x >= 0 else 0)

# 변환 결과 확인
print("Label Distribution")
print(data['Label'].value_counts())

# 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

# Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# SVM 모델 생성 및 학습
model = SVC(kernel='rbf', gamma='scale', class_weight=None, random_state=42)
model.fit(X_train, y_train)

# 검증 데이터 평가
y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)

print(f"Validation Accuracy: {val_accuracy:.2f}")
print(f"Validation F1 Score: {val_f1:.2f}")

# 테스트 데이터 평가
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)

print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Test F1 Score: {test_f1:.2f}")

# 분류 보고서 출력
print("\nClassification Report on Test Data:")
print(classification_report(y_test, y_test_pred))

Label Distribution
Label
1    19000
0     3000
Name: count, dtype: int64
Validation Accuracy: 0.89
Validation F1 Score: 0.94
Test Accuracy: 0.89
Test F1 Score: 0.94

Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.92      0.19      0.32       450
           1       0.89      1.00      0.94      2850

    accuracy                           0.89      3300
   macro avg       0.90      0.60      0.63      3300
weighted avg       0.89      0.89      0.85      3300



In [7]:
#필요한 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# 데이터 로드
data = pd.read_csv("monunmon.csv")  # 데이터셋 경로 입력
data['Label'] = data['Label'].apply(lambda x: 1 if x >= 0 else 0)  # 이진 레이블 변환

# 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

# Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

### 방법 1: 클래스 가중치 적용 ###
print("=== SVM with Class Weights ===")
model_weighted = SVC(random_state=42, class_weight='balanced')
model_weighted.fit(X_train, y_train)

# 테스트 데이터 평가
y_test_pred = model_weighted.predict(X_test)
print("\nClassification Report on Test Data (Class Weights):")
print(classification_report(y_test, y_test_pred))

### 방법 2: SMOTE를 이용한 오버샘플링 ###
print("\n=== SVM with SMOTE ===")
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("Resampled Training Set Size (SMOTE):", X_train_smote.shape)

model_smote = SVC(random_state=42)
model_smote.fit(X_train_smote, y_train_smote)

# 테스트 데이터 평가
y_test_pred_smote = model_smote.predict(X_test)
print("\nClassification Report on Test Data (SMOTE):")
print(classification_report(y_test, y_test_pred_smote))

### 방법 3: 언더샘플링 ###
print("\n=== SVM with Undersampling ===")
undersampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

print("Resampled Training Set Size (Undersampling):", X_train_under.shape)

model_under = SVC(random_state=42)
model_under.fit(X_train_under, y_train_under)

# 테스트 데이터 평가
y_test_pred_under = model_under.predict(X_test)
print("\nClassification Report on Test Data (Undersampling):")
print(classification_report(y_test, y_test_pred_under))

=== SVM with Class Weights ===

Classification Report on Test Data (Class Weights):
              precision    recall  f1-score   support

           0       0.36      0.74      0.48       450
           1       0.95      0.79      0.86      2850

    accuracy                           0.78      3300
   macro avg       0.65      0.76      0.67      3300
weighted avg       0.87      0.78      0.81      3300


=== SVM with SMOTE ===
Resampled Training Set Size (SMOTE): (26600, 15)

Classification Report on Test Data (SMOTE):
              precision    recall  f1-score   support

           0       0.37      0.74      0.49       450
           1       0.95      0.80      0.87      2850

    accuracy                           0.79      3300
   macro avg       0.66      0.77      0.68      3300
weighted avg       0.87      0.79      0.82      3300


=== SVM with Undersampling ===
Resampled Training Set Size (Undersampling): (4200, 15)

Classification Report on Test Data (Undersampling):
   